# Backward pass 

## Exercise e) Complete code for backward pass

Below is a implementation of the backward pass with some lines removed. Insert the missing lines.

#### Useful resources.

Math intro:
* https://www.deeplearningbook.org/contents/mlp.html
* http://neuralnetworksanddeeplearning.com/chap2.html
* http://pandamatak.com/people/anand/771/html/node37.html

Code intro:
* https://cs231n.github.io/neural-networks-case-study/#grad

<u>Attention!</u>

This is a difficult topic. It's normal to be confused and not immediately grasp the complete algorithm. Just work your way through it writing down the equations, writing down a small network explicitly, and thinking about the model.

#### Unpacking the backward pass.

Before we learned how to **forward** data input $X$ in a parametric model $W$ to obtain a final output $y$ and build a loss function $E(t, y)$ that measures how far our prediction $y$ is from the target $t$.
For a generic FFN at layer $l$ we can write:

$$a^{(l)} = W^{(l)} z^{(l-1)} + b^{(l)},$$
$$z^{(l)} = h(a^{(l)}),$$
where $z^{(0)}=x$ and $z^{(L)} = y$.


#### Backpropagation.

As the name suggests, the backpropagation algorithm is a procedure to adjust the model parameters $(W, b)$ 
<u>*propagating backward a measure of error*</u> such that our prediction $y$ is as close as possible to the target $t$.

The proxy we use to measure closeness between $y$ and $t$ is a loss function $E(t, y)$. 

<u>Attention!</u>
This loss is a function of the model parameters through $y = g(x, w, b)$. 
When we optimize wrt. the model parameters $(W, b)$ we can write the loss as a function of such parameters $E(w, b)$ because $y=g(w, b)$ and $t$ and $x$ are fixed input and output.

In practice, backpropagation is a gradient based optimization strategy. 
Our goal is to compute all the partial derivatives $\dfrac{\partial E(w, b)}{\partial w^l_{ji}}$ and $\dfrac{\partial E(w, b)}{\partial b^l_{j}}$ for each layer $l$ and each unit $i$ and $j$ in the network.

Computing the partial derivative wrt. the parameters directly is not easy. 
The loss depends in a non-linear and hierarchical way from the parameters. What backpropagation give us is an efficient and systematic way to compute the partial derivative as a function of intermediate quantities. The most important of such quantities is the error $\delta_l$.

#### Error propagation - final layer $L$.

Consider the final layer $L$.
The partial derivatives can be written as:

$$\dfrac{\partial E(w, b)}{\partial w^{L}_{ji}} =
\dfrac{\partial E}{\partial z^{L}_j}
\dfrac{\partial z^{L}_j}{\partial w^{L}_{ji}} = 
\left(
\color{blue}{\dfrac{\partial E}{\partial z^{L}_j}
\dfrac{\partial z^{L}_j}{\partial a^{L}_j}}
\right)
\dfrac{\partial a^{L}_j}{\partial w^{L}_{ji}} = 
\color{blue}{\delta^{L}_j}~\dfrac{\partial a^{L}_j}{\partial w^{L}_{ji}} = 
\delta^{L}_j~z^{L-1}_{i},
$$

where
$a^{L}_j = \sum_i w^{L}_{ji}~z^{L-1}_i + b^{L}_j$ and $z^{L}_j = h(a^{L}_j).$ 

$\delta^{L}_j = \partial E / \partial a^{L}_j$ is the error at layer $L$ for unit $j$ and it is what we want to backpropagate. 
You see that it can be easily computed: 
$ \partial E / \partial z^{L}_j$
is just the partial derivative of the loss wrt. to the activation.
$ \partial z^{L}_j / \partial a^{L}_j$
is the partial derivative of the activation wrt. the output of the affine transformation.
You already know how to compute these derivative (How?) and consequently you know $\delta^{L}_j$.

For the biases in the final layer the computation is basically the same:

$$\dfrac{\partial E(w, b)}{\partial b_{j}} =
\left(\dfrac{\partial E}{\partial z_j}
\dfrac{\partial z_j}{\partial a_j}\right)
\dfrac{\partial a_j}{\partial b_{j}} = 
\delta_j~\dfrac{\partial a_j}{\partial b_{j}} = 
\delta_j.
$$

You can see that $\delta_j$ is the same we computed, and the only thing that changes is the partial derivative wrt $w_{ji}$ and $b_j$.

#### Error propagation - layer $l$.

For a generic layer $l$ we want to derive a recursive formula for the error $\delta^{l}_j$.
In particular, if we at layer $l$ and unit $j$, we consider the $k$ downstream units influenced by $j$:

$$\dfrac{\partial E(w, b)}{\partial w^l_{ji}} =
\sum_k
\dfrac{\partial E}{\partial a^{l+1}_k}
\dfrac{\partial a^{l+1}_k}{\partial w^l_{ji}} =
\left(
\color{green}{\sum_k
\dfrac{\partial E}{\partial a^{l+1}_k}
\dfrac{\partial a^{l+1}_k}{\partial z^l_j}
\dfrac{\partial z^l_j}{\partial a^l_j}}
\right)
\dfrac{\partial a^l_j}{\partial w^l_{ji}} = 
\color{green}{\delta^{l}_j}~\dfrac{\partial a^{l}_j}{\partial w^{l}_{ji}} = 
\delta^l_j~z^{l-1}_{i},
$$

where $a^{l}_j = \sum_i w^{l}_{ji}~z^{l-1}_i + b^{l}_j$ and $z^{l}_j = h(a^{l}_j).$

Notice how the terms in $\delta^{l}_j$ are easy to compute: $\partial E  / \partial a^{l+1}_k = \delta^{l+1}_k$ is known (we start from the last layer). $\partial a^{l+1}_k / \partial z^{l}_j$ is something new, but we can easily compute this term too. 
We notice that:
$$a^{l+1}_k = \sum_j w^{l+1}_{kj} z^{l}_{j} + b_k,$$
and $\partial a^{l+1}_k / \partial z^{l}_j = w^{l+1}_{kj}$. 
Finally $\partial z^l_j / \partial a^l_j$ is just the partial derivative of the activation (as before).

We can now write the general recursive form for error propagation:

$$\delta^{l}_j = \left(\sum_k \delta^{l+1}_k w^{l+1}_{kj}\right) \dfrac{\partial z^l_j}{\partial a^l_j}$$

#### Putting it all together.

Here we are! Everything boils down to recursively finding the errors $\delta^{l}$ starting from the last layer $L$ backward. When you have $\delta^{l}$, you just need a final multiplication with activations to find the partial derivative wrt the parameters for the layer.
Let's write the steps in matrix form:

* Errror for the last layer $L$:
$$\delta^{L} = \nabla_z E \circ h'(a^{L}).$$
* Error for any layer $l$:
$$\delta^l = ((W^{l+1})^{T} \delta^{l+1}) \circ h'(a^{l}).$$
* Partial derivative cost for $w$: 
$$\dfrac{\partial E}{\partial w^{l}} = \delta^{l} z^{l-1}.$$
* Partial derivative cost for $b$:
$$\dfrac{\partial E}{\partial b^{l}} = \delta^{l}.$$

In [ ]:
def backward_pass(x, t, y, z, a, NN, activations, loss_f):
    """
    This function performs a backward pass ITERATIVELY. It saves lists all of the derivatives in the process
    
    Input:
    x:           The input used for the batch                (np.array)
    t:           The observed targets                        (np.array, the first dimension must be the same to x)
    y:           The output of the forward_pass of NN for x  (np.array, must have the same shape as t)
    
    a:           The affine transforms from the forward_pass (np.array) # a^{l+1}= W^{l} z^{l} + b^{l}
    z:           The activated units from the forward_pass (np.array)   # z^{l+1}=f(a^{l+1})
    
    activations: The activations to be used                  (list of functions)
    loss_f:        The loss function to be used                (one function)
    
    Output:
    g_w: A list of gradients for every weight
    g_b: A list of gradients for every bias
    
    Shapes for the einsum:
    b: batch size
    i: size of the input hidden layer (layer l)
    o: size of the output (layer l+1)
    """
    
    BS = x.shape[0] # Implied batch shape 
    
    # Process the last layer - Reference: Error propagation - final layer $L$.
    
    # First, let's compute the list of derivatives of z with respect to a
    # these derivative are standardized and automatically handled by the activations functions defined above.
    d_a = []
    # dz/da
    for i in range(len(activations)):
        d_za = activations[i](a[i], derivative=True)
        d_a.append(d_za)
    
    # Second, let's compute the derivative of the loss function with respect to z
    # targets
    t = t.reshape(BS, -1)
    
    # derivative loss wrt y
    # dE/dy  where y=z[-1]
    d_loss = 0      # <- Insert correct expression here
    
     
    # Third, let's compute the derivative of the biases and the weights
    g_w   = [] # List to save the gradient w.r.t. the weights
    g_b   = [] # List to save the gradients w.r.t. the biases
    
    # delta : measure of error in the final layer L
    # delta = dE/dy * dy/da
    delta = np.einsum('bo, bo -> bo', d_loss, d_a[-1]) # loss shape: (b, o); pre-activation units shape: (b, o) hadamard product
    
    # affine transformation
    # a = W z + b
    
    # dE/dw = (dE/dy * dy/da) da/dw = delta * da/dw
    # notice how the gradients wrt the weights have dimension (batch, input_dim, output_dim)
    g_w.append(np.mean(np.einsum('bo, bi -> bio', delta, z[-2]), axis=0)) # delta shape: (b, o), activations shape: (b, h)
    
    # dE/db = (dE/dy * dy/da) da/db = delta * da/db
    g_b.append(np.mean(delta, axis=0))
    
    
    # Process all the other layers - Reference: Error propagation - layer $l$
    for l in range(1, len(NN[0])):
        
        W = NN[0][-l] 
        # dE/dz^{l} = dE/da^{l+1} * da^{l+1}/dz^{l} = delta^{l+1} * w^{l+1}
        d_E_d_z = np.einsum('bo, io -> bi', delta, W)          # Derivative of the loss with respect to an activated layer d_E_d_z. 
                                                               #  delta shape: as above; weights shape: (i, o)
                                                               # Delta: d_E_d_z (element-wise mult) derivative of the activation layers
                                                               #  delta shape: as above; d_z shape: (b, i)  
        
        # delta : measure of error for a generic layer l 
        # dE/dz * dz/da 
        delta = 0      # <- Insert correct expression here 
        
        # affine transformation
        # a = Wz + b
        
        # dE/dw = delta * da/dw
        g_w.append(np.mean(np.einsum('bo, bi -> bio', delta, z[-l-2]), axis=0)) # Derivative of cost with respect to weights in layer l:
        
        # dE/db = delta                                                                 # delta shape: as above; activations of l-1 shape: (b, i)
        g_b.append(np.mean(delta, axis=0))
        
    return g_b[::-1], g_w[::-1]

# Backward pass unit test

We are going to perform the unit test of the backward pass with a finite difference estimation, make sure to read the description of the function and that you understand it well:

## Exercise f) Test correctness of derivatives with finite difference method

Write a small function that uses [the finite difference method](https://en.wikipedia.org/wiki/Finite_difference_method) to test whether the backpropation implementation is working. In short we will use
$$
\frac{\partial E(w)}{\partial w_{ij}^{(l)}} \approx \frac{E(v)-E(w)}{dw}
$$
for $dw \ll 1$ and $v$ is the same network as $w$ apart from $v_{ij}^{(l)} = w_{ij}^{(l)} + dw$.

As arguments the function should take: some data $x$ and $t$ as in the example above, the network including activations, the indices $i$, $j$, $l$ of the weight we investigate and $dw$ and return the right hand side of the expression above.

_Insert your code in the cell below._


In [ ]:
# Insert your finite difference code here
def finite_difference(x, t, NN, activations, indexes, dw=1e-10):
    """
    This function compute the finite difference between
    
    Input:
    x:           The input used for the batch                (np.array)
    t:           The observed targets                        (np.array, the first dimension must be the same to x)
    
    NN: The initialized neural network                       (tuple of list of matrices)
    activations: The activations to be used                  (list of functions)
    
    indexes: the indexes of the parameter we want to perturb (tuple of integers)
             v^{l}_{ji} = w^{l}_{ji} + dw
    
    dw: the size of the difference                           (float)
    
    Output:
    finite_difference: the magnitude of the difference       (float) 
    """
    
    from copy import deepcopy
    # l layer
    # i input dim
    # j output dim
    
    (l, i, j) = indexes
    
    _, zw = forward_pass(x, NN, activations)
    Ew=squared_error(t, zw[-1])
    
    NNv = deepcopy(NN)
    NNv[0][l][i, j] = 0 # <- Insert correct expression 
    
    _, zv = 0           # <- Insert correct expression
    Ev= 0               # <- Insert correct expression
    
    finite_difference = (Ev - Ew) / dw
    
    return finite_difference

Once you have implemented the function you can compare this number with the left hand side computed by the implementation above.

Try for different parameters and different values of $dw$. Scan over a range of $dw$ values. Why does the method break down for really small $dw$?

_Insert your written answer here._

Finite differences gives us gradients without computing gradients explicitly. Why don't we use it in practice then?

_Insert your written answer here._

Below is reference code that computes the finite differences for all parameters.

In [ ]:
def finite_diff_grad(x, NN, ACT_F, epsilon=None):
    """
    Finite differences gradient estimator: https://en.wikipedia.org/wiki/Finite_difference_method
    The idea is that we can approximate the derivative of any function (f) with respect to any argument (w) by evaluating the function at (w+e)
    where (e) is a small number and then computing the following opertion (f(w+e)-f(w))/e . Note that we would need N+1 evaluations of
    the function in order to compute the whole Jacobian (first derivatives matrix) where N is the number of arguments. The "+1" comes from the
    fact that we also need to evaluate the function at the current values of the argument.
    
    Input:
    x:       The point at which we want to evaluate the gradient
    NN:      The tuple that contains the neural network
    ACT_F:   The activation functions in order to perform the forward pass
    epsilon: The size of the difference
    
    Output:
    Two lists, the first one contains the gradients with respect to the weights, the second with respect to the biases
    """
    from copy import deepcopy
    
    if epsilon == None:
        epsilon = np.finfo(np.float32).eps # Machine epsilon for float 32
        
    grads = deepcopy(NN)               # Copy of structure of the weights and biases to save the gradients                        
    _ , test_z = forward_pass(x, NN_UT, ACT_F_UT) # We evaluate f(x)
    
    for e in range(len(NN)):                       # Iterator over elements of the NN:       weights or biases
        for h in range(len(NN[e])):                # Iterator over the layer of the element: layer number
            for r in range(NN[e][h].shape[0]):     # Iterator over                           row number
                for c in range(NN[e][h].shape[1]): # Iterator over                           column number 
                    NN_copy             = deepcopy(NN)    
                    NN_copy[e][h][r,c] += epsilon
                    _, test_z_eps       = forward_pass(x, NN_copy, ACT_F)     # We evaluate f(x+eps)
                    grads[e][h][r,c]    = (test_z_eps[-1]-test_z[-1])/epsilon # Definition of finite differences gradient
    
    return grads[0], grads[1]

In [ ]:
### Unit test 

## First lest's compute the backward pass using our own function
# Forward pass
test_a, test_z = forward_pass(np.array([[1,1,1]]), NN_UT, ACT_F_UT)
# Backward pass
test_g_b, test_g_w = backward_pass(np.array([[1,1,1]]), np.array([20]), test_a[-1], test_z, test_a, NN_UT, ACT_F_UT, squared_error)
# Estimation by finite differences
test_fdg_w, test_fdg_b = finite_diff_grad(np.array([[1,1,1]]), NN_UT, ACT_F_UT)

In [ ]:
# Test whether the weights and biases are all equal as the ones we estimated using back propagation
for l in range(len(test_g_w)):
    assert np.allclose(test_fdg_w[l], test_g_w[l])
    assert np.allclose(test_fdg_b[l], test_g_b[l])

# Training and validation

We are ready to train some neural networks! Below we give some example initializations and a training loop. Try it out. 

In [ ]:
# Initialize an arbitrary neural network
#L  = [3, 16, 1]
L  = [1, 8, 1]
NN = init_NN(L)
#NN = init_NN_glorot(L, uniform=True)
#NN = init_NN_he_ReLU(L, uniform=True)

ACT_F = [ReLU, Linear]
#ACT_F = [Tanh, Linear]

# Recommended hyper-parameters for 1-D: 
# L  = [1, 8, 1]
# EPOCHS = 10000
# BATCH_SIZE = 128 
# LEARN_R = 2.5e-1 for Tanh and LEARN_R = 1e-1 for ReLU

# Recommended hyper-parameters for 3-D: 
# L  = [3, 16, 1] 
# EPOCHS = 10000
# BATCH_SIZE = 128 
# LEARN_R = 5e-2 for ReLU and LEARN_R = 1e-1 for Tanh

### Notice that, when we switch from tanh to relu activation, we decrease the learning rate. This is due the stability of the gradients 
## of the activation functions.

In [ ]:
# Initialize training hyperparameters
EPOCHS = 20000
BATCH_SIZE = 128 
LEARN_R = 1e-2 

In [ ]:
train_loss = []
val_loss = []

for e in range(EPOCHS):
    # Mini-batch indexes
    idx = np.random.choice(x_train.shape[0], size=BATCH_SIZE)
    # Forward pass
    aff, units = forward_pass(x_train[idx,:], NN, ACT_F)
    # Backward pass
    g_b, g_w = backward_pass(x_train[idx,:], y_train[idx], units[-1], units, aff, NN, ACT_F, squared_error)
    
    # Stochastic gradient descent
    for l in range(len(g_b)):
        NN[0][l] -= LEARN_R*g_w[l]
        NN[1][l] -= LEARN_R*g_b[l]
        
    # Training loss
    _, units = forward_pass(x_train, NN, ACT_F)
    # Estimate loss function
    #print(np.max(squared_error(y_train, units[-1])))
    train_loss.append(np.mean(squared_error(y_train, np.squeeze(units[-1]))))
    
    # Validation
    # Forward pass
    _, units = forward_pass(x_validation, NN, ACT_F)
    # Estimate validation loss function
    val_loss.append(np.mean(squared_error(y_validation, np.squeeze(units[-1]))))
    
    if e%500==0:
        print("{:4d}".format(e),
              "({:5.2f}%)".format(e/EPOCHS*100), 
              "Train loss: {:4.3f} \t Validation loss: {:4.3f}".format(train_loss[-1], val_loss[-1]))
        


In [ ]:
plt.plot(range(len(train_loss)), train_loss);
plt.plot(range(len(val_loss)), val_loss);

# Testing

We have kept the calculation of the test error separate in order to emphasize that you should not use the test set in optimization.

In [ ]:
_, units = forward_pass(x_test, NN, ACT_F)

In [ ]:
plt.scatter(y_test, units[-1]);
plt.plot([np.min(y_test), np.max(y_test)], [np.min(y_test), np.max(y_test)], color='k');
plt.xlabel("y");
plt.ylabel("$\hat{y}$");
plt.title("Model prediction vs real in the test set, the close to the line the better")
plt.grid(True);
plt.axis('equal');
plt.tight_layout();

print("Test loss:  {:4.3f}".format(np.mean(squared_error(y_test, np.squeeze(units[-1])))))

In [ ]:
if D1:
    plt.scatter(x_train[:,0], y_train, label="train data");
    plt.scatter(x_test[:,0], units[-1], label="test prediction");
    plt.scatter(x_test[:,0], y_test, label="test data");
    plt.legend();
    plt.xlabel("x");
    plt.ylabel("y");
else:
    plt.scatter(x_train[:,1], y_train, label="train data");
    plt.scatter(x_test[:,1], units[-1], label="test data prediction");
    plt.scatter(x_test[:,1], y_test, label="test data");
    plt.legend();
    plt.xlabel("x");
    plt.ylabel("y");

## Exercise g) Show overfitting, underfitting and just right fitting

Vary the architecture and other things to show clear signs of overfitting (=training loss significantly lower than test loss) and underfitting (=not fitting enoung to training data so that test performance is also hurt).

See also if you can get a good compromise which leads to a low validation loss. 

For this problem do you see any big difference between validation and test loss? The answer here will probably be no. Discuss cases where it is important to keep the two separate.

_Insert written answer here._


In [ ]:
# Insert your code for getting overfitting, underfitting and just right fitting

# Next steps - classification

It is straight forward to extend what we have done to classification. 

For numerical stability it is better to make softmax and cross-entropy as one function so we write the cross entropy loss as a function of the logits we talked about last week. 

Next week we will see how to perform classification in PyTorch.

## Exercise h) optional - Implement backpropagation for classification

Should be possible with very few lines of code. :-)

In [ ]:
# Just add code.